In [ ]:
# Downloading pyspark
#!pip install pyspark

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

PREPARE THE ENVIRONMENT, UPLOAD DATA, PREPROCESS DATA AND CREATE THE TABLES: Author, Paper, Affiliation, Book, Journal and Conference

In [1]:
# With sparkSession we create a connection to our database
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col, xxhash64, collect_list, explode
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, TimestampType

# Create an entry point to the PySpark Application
spark = SparkSession.builder \
    .master("local") \
    .appName("Bibliography") \
    .getOrCreate()

INPUT_FILE = "bibliography.json"
OPTIONS = {'multiline': 'true', 'allowNumericLeadingZero': 'true', 'timestampFormat': "yyyy-MM-dd'T'HH:mm:ss[.ZZZ'Z']"}

22/12/11 19:33:50 WARN Utils: Your hostname, MacBook-Air-di-Enrico.local resolves to a loopback address: 127.0.0.1; using 172.20.10.13 instead (on interface en0)
22/12/11 19:33:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/11 19:33:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
#AUTHOR TABLE
schemaAut = StructType(
    [StructField('authors', ArrayType(StructType([
        StructField('_id', StringType(), nullable=False),
        StructField('name', StringType(), True),
        StructField('email', StringType(), True),
        StructField('bio', StringType(), True),
    ])), True)
     ])

df_aut = spark.read.format('json').options(**OPTIONS).schema(schemaAut).json(INPUT_FILE)
df_aut = df_aut.select(explode(df_aut.authors))
df_aut = df_aut.withColumnRenamed("col", "authors")
df_aut = df_aut.filter(col("authors._id") != "null").select("authors._id", "authors.name", "authors.email",
                                                            "authors.bio")
df_aut = df_aut.withColumnRenamed("_id", "author_id")
df_aut = df_aut.dropDuplicates(["author_id"])
df_aut.printSchema()
df_aut.show()

root
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- bio: string (nullable = true)



+--------------------+------------------+--------------------+--------------------+
|           author_id|              name|               email|                 bio|
+--------------------+------------------+--------------------+--------------------+
|53f3186fdabfae9a8...|   A. M. A. Hariri|a..m..a..hariride...|My name is A. M. ...|
|53f3186fdabfae9a8...|    Matthew Prowse|matthew.prowsefb@...|My name is Matthe...|
|53f31870dabfae9a8...|       Sui-ping Qi|sui-ping.qi19@gma...|My name is Sui-pi...|
|53f31871dabfae9a8...|     Renato Fabbri|renato.fabbrib7@g...|My name is Renato...|
|53f31873dabfae9a8...|   Joachim Schimpf|joachim.schimpf8a...|My name is Joachi...|
|53f31874dabfae9a8...|    E. Di Bernardo|e..di.bernardo10@...|My name is E. Di ...|
|53f31875dabfae9a8...|    Steven F. Roth|steven.f..roth46@...|My name is Steven...|
|53f31878dabfae9a8...|      Nima Zahadat|nima.zahadat3d@gm...|My name is Nima Z...|
|53f3187ddabfae9a8...|         Ke Fa Cen|ke.fa.cen23@gmail...|My name is Ke 

In [3]:
# PAPER TABLE WITHOUT PUBLICATION_ID
schemaPaper = StructType(
    [StructField('_id', StringType(), True),
     StructField('title', StringType(), True),
     StructField('keywords', ArrayType(StringType()), True),
     StructField('fos', ArrayType(StringType()), True),
     StructField('references', ArrayType(StringType()), True),
     StructField('page_start', IntegerType(), True),
     StructField('page_end', IntegerType(), True),
     StructField('lang', StringType(), True),
     StructField('abstract', StringType(), True),
     StructField('publication_type', StringType(), True),
     StructField('date', TimestampType(), True),
     StructField('doi', StringType(), True),
     StructField('url', ArrayType(StringType()), True)
     ])

df_paper = spark.read.format('json').options(**OPTIONS).schema(schemaPaper).json(INPUT_FILE)
df_paper = df_paper.withColumnRenamed("_id", "paper_id")
df_paper.printSchema()
df_paper.show()

root
 |-- paper_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- fos: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- references: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- page_start: integer (nullable = true)
 |-- page_end: integer (nullable = true)
 |-- lang: string (nullable = true)
 |-- abstract: string (nullable = true)
 |-- publication_type: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- doi: string (nullable = true)
 |-- url: array (nullable = true)
 |    |-- element: string (containsNull = true)



+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------+----+--------------------+----------------+-------------------+--------------------+--------------------+
|            paper_id|               title|            keywords|                 fos|          references|page_start|page_end|lang|            abstract|publication_type|               date|                 doi|                 url|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------+----+--------------------+----------------+-------------------+--------------------+--------------------+
|53e99784b7602d970...|Using XML to Inte...|[internet, hyperm...|[xml base, world ...|[53e9adbdb7602d97...|       167|     172|  en|The eXtensible Ma...|            Book|1974-09-13 06:34:29|10.1109/CMPSAC.20...|[http://dx.doi.or...|
|53e99784b7602d970...|               FCLOS|[molap, subsumpti...|[informa

In [4]:
# AFFILIATION TABLE
schemaAffiliation = StructType(
    [StructField('_id', StringType(), True),
     StructField('authors', ArrayType(StructType([
         StructField('_id', StringType(), True),
         StructField('org', StringType(), True)
     ])), True),
     ])

df_aff = spark.read.format('json').options(**OPTIONS).schema(schemaAffiliation).json(INPUT_FILE)
df_aff = df_aff.withColumnRenamed("_id", "paper_id")
df_aff = df_aff.select("paper_id", explode(df_aff.authors))
df_aff = df_aff.withColumnRenamed("col", "authors")
df_aff = df_aff.filter(col("authors._id") != "null").filter(col("paper_id") != "null").select("paper_id", "authors._id",
                                                                                              "authors.org")
df_aff = df_aff.withColumnRenamed("_id", "author_id")
df_aff = df_aff.dropDuplicates(["author_id", "paper_id"])
df_aff = df_aff.withColumnRenamed("org", "organization")
df_aff.printSchema()
df_aff.show()

root
 |-- paper_id: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- organization: string (nullable = true)



+--------------------+--------------------+--------------------+
|            paper_id|           author_id|        organization|
+--------------------+--------------------+--------------------+
|53e998c7b7602d970...|53f3186fdabfae9a8...|Department of Sta...|
|53e99827b7602d970...|53f3186fdabfae9a8...|Laboratory for Fo...|
|53e99924b7602d970...|53f31870dabfae9a8...|Henan Academy of ...|
|53e998dbb7602d970...|53f31871dabfae9a8...|Instituto de Físi...|
|53e998f6b7602d970...|53f31873dabfae9a8...|                null|
|53e998bfb7602d970...|53f31874dabfae9a8...|                null|
|53e9984bb7602d970...|53f31875dabfae9a8...|                null|
|53e998e8b7602d970...|53f31878dabfae9a8...|George Mason Univ...|
|53e99905b7602d970...|53f3187ddabfae9a8...|State Key Laborat...|
|53e998e9b7602d970...|53f31881dabfae9a8...|                null|
|53e9984fb7602d970...|53f31881dabfae9a8...|Tecnologico de Mo...|
|53e9980eb7602d970...|53f31883dabfae9a8...|University of Was...|
|53e997e9b7602d970...|53f

In [5]:
# JOURNAL TABLE
# Preprocessing of the journals for cleaning and merging the journals

journal_schema_preprocessing = StructType(
    [StructField('_id', StringType(), True),
     StructField('issn', StringType(), True),
     StructField('publisher', StringType(), True),
     StructField('venue', StringType(), True),
     StructField('volume', IntegerType(), True),
     StructField('issue', IntegerType(), True),
     StructField('publication_type', StringType(), True)])

# Reading the json file
df_journals_to_filter = spark.read.format('json').options(**OPTIONS).schema(journal_schema_preprocessing).json(
    INPUT_FILE)

# Filtering and adjusting the dataframe
df_journals_to_filter = df_journals_to_filter.filter(col('publication_type') == 'Journal').filter(
    col('issn') != 'null').filter(col('venue') != 'null').filter(col('issue') >= 0).filter(col('volume') >= 0)
df_journals_to_filter = df_journals_to_filter.groupBy('venue', 'volume', 'issue', 'issn').agg(
    collect_list('publisher').alias('publishersArray'), collect_list('_id').alias('_id'),
    count(col('publisher')))  # count can be removed (I was interested in evaluating if the group by was meaningful)
df_journals_to_insert = df_journals_to_filter.withColumn('publisher',
                                                         df_journals_to_filter['publishersArray'][0]).select('venue',
                                                                                                             'volume',
                                                                                                             'issue',
                                                                                                             'publisher',
                                                                                                             'issn',
                                                                                                             '_id')

# Adding the new column which contains the publication_identifier
df_journals = df_journals_to_insert.withColumn("publication_id", xxhash64('venue', 'volume', 'issue', 'issn'))

# Adding the foreign key to the papers
exploded_journals = df_journals.select(explode('_id'), 'publication_id')
#exploded_journals.show(truncate = False)

df_papers_in_journals = exploded_journals.join(df_paper, exploded_journals.col == df_paper.paper_id, "inner")
df_papers_in_journals = df_papers_in_journals.drop('col')

df_journals = df_journals.drop(df_journals._id)

# Visualizing the data
#print('Papers')
#df_papers_in_journals.show(truncate = False)
print('Schema of the journals')
df_journals.printSchema()
print('Journals')
df_journals.show(truncate=False)

Schema of the journals
root
 |-- venue: string (nullable = true)
 |-- volume: integer (nullable = true)
 |-- issue: integer (nullable = true)
 |-- publisher: string (nullable = true)
 |-- issn: string (nullable = true)
 |-- publication_id: long (nullable = false)

Journals
+-----------------------+------+-----+------------------------------------------------------------------+---------+--------------------+
|venue                  |volume|issue|publisher                                                         |issn     |publication_id      |
+-----------------------+------+-----+------------------------------------------------------------------+---------+--------------------+
|4OR                    |2     |4    |Inderscience Publishers                                           |1614-2411|-8668689828003778653|
|4OR                    |3     |1    |Accent Social and Welfare Society                                 |1614-2411|-6055461944651662439|
|4OR                    |4     |1    |Inn

In [6]:
# BOOK TABLE
# Preprocessing of the books for cleaning and merging the books
book_schema_preprocessing = StructType(
    [StructField('_id', StringType(), True),
     StructField('isbn', StringType(), True),
     StructField('publisher', StringType(), True),
     StructField('venue', StringType(), True),
     StructField('publication_type', StringType(), True)])

# Reading the json file
df_books_to_filter = spark.read.format('json').options(**OPTIONS).schema(book_schema_preprocessing).json(INPUT_FILE)

# Filtering and adjusting the dataframe
df_books_to_filter = df_books_to_filter.filter(col('publication_type') == 'Book').filter(col('isbn') != 'null').filter(
    col('venue') != 'null')
df_books_to_filter = df_books_to_filter.groupBy('isbn', 'venue').agg(collect_list('publisher').alias('publishersArray'),
                                                                     collect_list('_id').alias('_id'), count(
        col('publisher')))  # count can be removed (I was interested in evaluating if the group by was meaningful)
dfbooks_to_insert = df_books_to_filter.withColumn('publisher', df_books_to_filter['publishersArray'][0]).select('venue',
                                                                                                                'isbn',
                                                                                                                'publisher',
                                                                                                                '_id')

# Adding the new column which is the id
df_books = dfbooks_to_insert.withColumn('publication_id', xxhash64('isbn', 'venue'))

# Adding the foreign key to the papers
exploded_books = df_books.select(explode('_id'), 'publication_id')
# exploded_books.show(truncate = False)

df_papers_in_books = exploded_books.join(df_paper, exploded_books.col == df_paper.paper_id)
df_papers_in_books = df_papers_in_books.drop('col')

df_books = df_books.drop(df_books._id)

# Visualizing the data
# print('Papers')
# df_papers_in_books.show(truncate = False)
print('Schema of the books')
df_books.printSchema()
print('Books')
df_books.show(truncate=40)

Schema of the books
root
 |-- venue: string (nullable = true)
 |-- isbn: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- publication_id: long (nullable = false)

Books
+----------------------------------------+-------------+----------------------------------------+--------------------+
|                                   venue|         isbn|                               publisher|      publication_id|
+----------------------------------------+-------------+----------------------------------------+--------------------+
|  ACM SIGSOFT Software Engineering Notes|-159593-125-2|AGH University of Science and Technology|-4146681027596907540|
|                     Theor. Comput. Sci.|  0-0304-3975|                                Elsevier|-8641604748836216424|
|                         ACL (Companion)| 0-111-456789|                          Intellect Ltd.|-1765624198251536382|
|                                    VLDB|0-12-088469-0|The American Society of Mechanical En..

In [7]:
# CONFERENCE TABLE
# Preprocessing of the books for cleaning and merging the books

schemaConf = StructType(
    [StructField('_id', StringType(), True),
     StructField('location', StringType(), True),
     StructField('venue', StringType(), True),
     StructField('publication_type', StringType(), True)])
# Reading the json file
df_conferences_to_filter = spark.read.format('json').options(**OPTIONS).schema(schemaConf).json(INPUT_FILE)

# Filtering and adjusting the dataframe
df_conferences_to_filter = df_conferences_to_filter \
    .filter(col('publication_type') == 'Conference') \
    .filter(col('venue') != 'null')
# count can be removed (I was interested in evaluating if the group by was meaningful)
df_conferences_to_filter = df_conferences_to_filter \
    .groupBy('venue') \
    .agg(collect_list('location').alias('locations_array'), collect_list('_id').alias('_id'), count(col('location')))
df_conferences_to_insert = df_conferences_to_filter \
    .withColumn('location', df_conferences_to_filter['locations_array'][0]) \
    .select('venue', 'location', '_id')

# Adding the new column which is the id
df_conferences = df_conferences_to_insert.withColumn('publication_id', xxhash64('venue'))

# Adding the foreign key to the papers
exploded_conferences = df_conferences.select(explode('_id'), 'publication_id')
#exploded_conferences.show(truncate = False)

df_papers_in_conferences = exploded_conferences.join(df_paper, exploded_conferences.col == df_paper.paper_id)
df_papers_in_conferences = df_papers_in_conferences.drop('col')

df_conferences = df_conferences.drop(df_conferences._id)

# Visualizing the data
#print('Papers')
#df_papers_in_conferences.show(truncate = False)
print('Schema of the conferences')
df_conferences.printSchema()
print('Conferences')
df_conferences.show(truncate=50)

Schema of the conferences
root
 |-- venue: string (nullable = true)
 |-- location: string (nullable = true)
 |-- publication_id: long (nullable = false)

Conferences
+--------------------------------------------------+--------------------------+--------------------+
|                                             venue|                  location|      publication_id|
+--------------------------------------------------+--------------------------+--------------------+
|                                           "EDUCON|            Moscow, Russia|  950373860555954453|
|2012 50TH ANNUAL ALLERTON CONFERENCE ON COMMUNI...|           Dublin, Ireland|-4245717996156385657|
|                                        2985415099|       Mexico City, Mexico| -762166203857654039|
|                                        2985532720|              Milan, Italy|-7124098142925130015|
|                                        2987493177|              Osaka, Japan| 7551243357793819285|
|                         

In [8]:
# Merging the 3 dataframe which one contains the papers published in a specific media
df_papers = df_papers_in_books \
    .union(df_papers_in_journals) \
    .union(df_papers_in_conferences)

# Visualizing the data
print('Papers schema')
df_papers.printSchema()
print('Papers data')
df_papers.show(truncate=False)

Papers schema
root
 |-- publication_id: long (nullable = false)
 |-- paper_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- keywords: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- fos: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- references: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- page_start: integer (nullable = true)
 |-- page_end: integer (nullable = true)
 |-- lang: string (nullable = true)
 |-- abstract: string (nullable = true)
 |-- publication_type: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- doi: string (nullable = true)
 |-- url: array (nullable = true)
 |    |-- element: string (containsNull = true)

Papers data


+--------------------+------------------------+-------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------

In [9]:
# For checking the result
print('Papers published in books')
df_papers \
    .filter(col('publication_type') == 'Book') \
    .select('paper_id', 'title', 'publication_type', 'publication_id') \
    .show()
print('Papers published in conferences')
df_papers \
    .filter(col('publication_type') == 'Conference') \
    .select('paper_id', 'title', 'publication_type', 'publication_id') \
    .show()


print('Papers published in journals')
df_papers \
    .filter(col('publication_type') == 'Journal') \
    .select('paper_id', 'title', 'publication_type', 'publication_id') \
    .show(truncate = False)

Papers published in books


+--------------------+--------------------+----------------+--------------------+
|            paper_id|               title|publication_type|      publication_id|
+--------------------+--------------------+----------------+--------------------+
|53e997d1b7602d970...|A fault diagnosis...|            Book|-4336127124407324726|
|53e997e4b7602d970...|Problem Decomposi...|            Book| 4674035537543784623|
|53e997e8b7602d970...|X-tract: Structur...|            Book| 1144530160964015551|
|53e997e8b7602d970...|Cognitive agent p...|            Book|-6465152039831803131|
|53e997e9b7602d970...|Constraint based ...|            Book| 5112790730981969721|
|53e997ecb7602d970...|Automatic input r...|            Book| 1369111316993402587|
|53e997ecb7602d970...|Acceptability-ori...|            Book| 1430384888819899512|
|53e997ecb7602d970...|Anomalous Neighbo...|            Book|  710748220667054414|
|53e997f1b7602d970...|A Digital Watermark.|            Book| 8148044123726554362|
|53e997f4b7602d9

+--------------------+--------------------+----------------+--------------------+
|            paper_id|               title|publication_type|      publication_id|
+--------------------+--------------------+----------------+--------------------+
|53e99854b7602d970...|  The EOLES project.|      Conference|  950373860555954453|
|53e9989bb7602d970...|Life is engineeri...|      Conference|  950373860555954453|
|53e998bfb7602d970...|Gaining and maint...|      Conference|  950373860555954453|
|53e998c0b7602d970...|From manuals towa...|      Conference|  950373860555954453|
|53e998e9b7602d970...|Learning with com...|      Conference|  950373860555954453|
|53e99976b7602d970...|Cloud E-learning ...|      Conference|  950373860555954453|
|53e9997eb7602d970...|Motivating progra...|      Conference|  950373860555954453|
|53e99991b7602d970...|Monitoring studen...|      Conference|  950373860555954453|
|53e99998b7602d970...|OLAREX project: O...|      Conference|  950373860555954453|
|53e99859b7602d9

+------------------------+-----------------------------------------------------------------------------------------------------------+----------------+--------------------+
|paper_id                |title                                                                                                      |publication_type|publication_id      |
+------------------------+-----------------------------------------------------------------------------------------------------------+----------------+--------------------+
|53e99915b7602d97021502bb|A note on robust 0-1 optimization with uncertain cost coefficients                                         |Journal         |-8668689828003778653|
|53e99984b7602d97021c6a1c|A new approach for unsupervised classification                                                             |Journal         |-6055461944651662439|
|53e998b0b7602d97020ebee5|Two-machine flow shop scheduling problems with minimal and maximal delays                                  |J

COMMANDS

In [15]:
# Command 1: Add a new row to the Paper DataFrame
from pyspark.sql.functions import lit

new_paper_file = 'single_paper.json'

new_paper = spark.read.options(**OPTIONS).json(new_paper_file, schemaPaper) \
    .withColumnRenamed("_id", "paper_id")

journal_schema = StructType(
    [StructField('issn', StringType(), True),
     StructField('publisher', StringType(), True),
     StructField('venue', StringType(), True),
     StructField('volume', IntegerType(), True),
     StructField('issue', IntegerType(), True)])

journal = spark.read.options(**OPTIONS).json(new_paper_file, journal_schema) \
    .withColumn("publication_id", xxhash64('venue', 'volume', 'issue', 'issn'))

foreign_key = journal.head(1)[0].asDict()['publication_id']

if df_journals.filter(col('publication_id') == foreign_key).count() == 0:
    df_jounrals = df_journals.union(journal)

paper_id = new_paper.head(1)[0].asDict()['paper_id']

# Inserting foreign_key at position 1 (0 for programmers)
new_paper = new_paper.select(lit(foreign_key).alias('publication_id'), '*')

if df_papers.filter(col('paper_id') == paper_id).collect() == []:
    df_papers = df_papers.union(new_paper)

df_papers.filter(col('paper_id') == paper_id).show(vertical=True)

-RECORD 0--------------------------------
 publication_id   | 6935858940620736824  
 paper_id         | 53e9999eb7602d970... 
 title            | Performance evalu... 
 keywords         | [system level per... 
 fos              | [reconfigurabilit... 
 references       | [53e9b47cb7602d97... 
 page_start       | 1656                 
 page_end         | 1667                 
 lang             | en                   
 doi              | 10.1155/ASP.2005.... 
 url              | [http://dx.doi.or... 
 abstract         | A reconfigurable ... 
 publication_type | Journal              
 date             | 1998-07-15 06:56:20  



In [14]:
# Update one single row of a dataframe (or multiple rows)
# The command modifies the DOI and URL of a paper with identifier equal to '53e997e4b7602d9701fdb48a'.
# For doing this operation we firstly filter the dataframe keeping only the row to be modified. We add a column for each value we want to insert under the name new_fieldName. Then we drop the old columns containing the previous values and we rename the new columns with the name of the old ones.
#Finally, we make the union between the entire dataframe, without the row we want to modify, and the new entry.
from pyspark.sql.functions import lit, array

updated_df_papers = df_papers\
    .filter(col('paper_id') == '53e997e4b7602d9701fdb48a')
updated_df_papers = updated_df_papers\
    .withColumn('new_doi', lit('10.1007/11944577_37'))\
    .withColumn('new_url', array([lit('https://link.springer.com/chapter/10.1007/11944577_37')]))
updated_df_papers = updated_df_papers\
    .drop(col('doi')).drop(col('url'))\
    .withColumnRenamed('new_doi', 'doi')\
    .withColumnRenamed('new_url', 'url')

# To check the number of the paper is the same as before
updated_df_papers = df_papers.filter(col('paper_id') != '53e997e4b7602d9701fdb48a').union(updated_df_papers)
print('The size of the entire initial database is ' + str(df_papers.count()) + ', the size of the current database is ' + str(updated_df_papers.count()))
updated_df_papers.filter(col('paper_id') == '53e997e4b7602d9701fdb48a').select('paper_id', 'title', 'doi', 'url').show(truncate = 55)

The size of the entire initial database is 37626, the size of the current database is 37626


+------------------------+-------------------------------------------------------+-------------------+-------------------------------------------------------+
|                paper_id|                                                  title|                doi|                                                    url|
+------------------------+-------------------------------------------------------+-------------------+-------------------------------------------------------+
|53e997e4b7602d9701fdb48a|CitizenTalk: application of chatbot infotainment to ...|10.1007/11944577_37|[https://link.springer.com/chapter/10.1007/11944577_37]|
+------------------------+-------------------------------------------------------+-------------------+-------------------------------------------------------+



In [ ]:
# Command 3: remove an entire column
df_papers_without_lang = df_papers \
    .drop('lang')

df_papers_without_lang.printSchema()
df_papers_without_lang.show(1)

In [ ]:
# Command 4: delete a group of rows

from pyspark.sql.functions import year

df_papers = df_papers \
    .filter(year('date') > '1950')

df_papers.select('title', 'publication_type', 'date') \
    .orderBy('date') \
    .show(truncate=False)

In [ ]:
# Command 5: create a new column with the length of the paper (number of total pages)

df_papers_total_pages = df_papers \
    .filter((col('page_start') >= 0) & (col('page_end') >= 0) & (col('page_start') <= col('page_end'))) \
    .withColumn('total_pages', col('page_end') - col('page_start'))

df_papers_total_pages \
    .select(col('title'), col('page_start'), col('page_end'), col('total_pages')) \
    .show(5, truncate=False)

QUERIES

In [ ]:
#Query 1: WHERE, JOIN

venue, volume, issue = ('BMC Bioinformatics', '14', '1')

df_papers_q1 = df_journals \
    .filter((col('venue') == venue) &
            (col('volume') == volume) &
            (col('issue') == issue)) \
    .join(df_papers,
          (df_journals['publication_id'] == df_papers['publication_id']) &
          (df_papers['publication_type'] == 'Journal'))

df_papers_q1 \
    .select(['paper_id', 'title']) \
    .show(truncate=60)

In [18]:
# Query 2: WHERE, LIMIT, LIKE
# Find the papers written in the last twenty years in english whose keywords have the word \verb|artificial| inside the keywords. We require that these papers have the DOI set to a not null value.
# The results are ordered ascending by the date and only 20 elements are printed.

from pyspark.sql.functions import current_timestamp, unix_timestamp

df = df_papers.withColumn('current time', current_timestamp())
df \
    .filter((((unix_timestamp('current time') - unix_timestamp('date')) / 3600 / 24 / 365) > 50) &
            (col('doi').isNotNull())) \
    .select('title',
            'date',
            explode('keywords').alias('keyword')) \
    .filter(col('keyword').like('%artificial%')) \
    .distinct() \
    .sort(col('date').asc()) \
    .limit(15) \
    .show(truncate=50)

+--------------------------------------------------+-------------------+----------------------------------------+
|                                             title|               date|                                 keyword|
+--------------------------------------------------+-------------------+----------------------------------------+
|An Ensemble Approach of Simple Regression Model...|1950-03-26 17:54:02|                 artificial intelligence|
|Structure Learning of Bayesian Networks Based o...|1950-03-28 09:32:11|        learning artificial intelligence|
|                         Robot social intelligence|1950-03-28 09:32:11|                 artificial intelligence|
|A Hybrid Formal Theory of Plan Recognition and ...|1950-05-04 20:32:22|                  artificial intelligent|
|Financial risk evaluation of chinese commercial...|1950-05-04 20:32:22|              artificial neural networks|
|Backpropagation Algorithm for Logic Oriented Ne...|1950-07-09 17:07:10|               a

In [ ]:
# Query 3: WHERE, IN, Nested Query
# Show the papers collected in a book that have `multiagent system` as keyword.

nested_query = df_papers \
    .select('title',
            'publication_type',
            'paper_id',
            'publication_id',
            'date',
            explode('keywords').alias('keyword')) \
    .filter(col('keyword').isin('multiagent system')) \
    .drop('keyword')
df_conferences \
    .join(nested_query, 'publication_id') \
    .filter(col('publication_type') == 'Conference') \
    .orderBy('date', ascending=False) \
    .select(col('title').alias('paper title'),
            col('venue').alias('conference venue')) \
    .show(5, truncate=50)

In [ ]:
# Query 4: GROUP BY, JOIN, AS

from pyspark.sql.functions import collect_set, size

df_aff \
    .join(df_papers, df_papers.paper_id == df_aff.paper_id, 'inner') \
    .drop(df_papers.paper_id) \
    .select('paper_id',
            'organization',
            'publication_type') \
    .filter((col('organization').isNotNull()) &
            (col('organization') != "") &
            (col('publication_type') == "Conference")) \
    .groupBy('organization') \
    .agg(collect_set('paper_id').alias('papers')) \
    .filter(size(col('papers')) > 10) \
    .show(truncate=50)

In [ ]:
# Query 5: WHERE, GROUP BY
# Retrieve some statistics about papers

from pyspark.sql.functions import sum, min, max, avg, format_number, variance

df_papers_total_pages \
    .filter(year(col('date')) >= 2015) \
    .groupBy(year(col('date')).alias('year')) \
    .agg(count('paper_id').alias('total_papers'),
         sum('total_pages').alias('total_pages'),
         min('total_pages').alias('min_pages'),
         max('total_pages').alias('max_pages'),
         format_number(avg('total_pages'), 2).alias('avg_pages'),
         format_number(variance('total_pages'), 2).alias('var_pages')) \
    .sort(col('year').desc()) \
    .show()

In [ ]:
# Query 6: GROUP BY, HAVING, AS

df_papers \
    .select('paper_id',
            'title',
            explode(col('references')).alias('reference')) \
    .groupBy('reference') \
    .agg(count('paper_id').alias('references_count')) \
    .filter(col('references_count') > 30) \
    .join(df_papers, col('reference') == df_papers.paper_id) \
    .sort(col('references_count').desc()) \
    .select(['title', 'references_count']) \
    .show(truncate=50)

In [24]:

# Query 7: WHERE, GROUP BY, HAVING, AS
# The query returns the association between fields of study and keywords which are more present within the database and how many times they appear together
from pyspark.sql.functions import year, col, lit, size

df = df_papers
df = df.withColumn('count', lit(1))
df = df \
    .filter(
    (col('doi').isNotNull()) &
    (year(col('date')) >= 2000) &
    (size(col('fos')) > 0) &
    (size(col('keywords')) > 0)) \
    .select('fos', 'count', explode('keywords').alias('keyword')) \
    .select('keyword', explode('fos').alias('fos'), 'count') \
    .groupby('fos', 'keyword') \
    .sum('count') \
    .withColumnRenamed('sum(count)', 'couple count') \
    .filter(col('couple count') > 100) \
    .sort(col('couple count').desc()) \
    .limit(15) \
    .show(truncate=False)

+-------------------------------+--------------------------------+------------+
|fos                            |keyword                         |couple count|
+-------------------------------+--------------------------------+------------+
|computer science               |internet                        |247         |
|computer science               |data mining                     |242         |
|computer science               |computer science                |226         |
|computer science               |real time                       |189         |
|computer science               |protocols                       |178         |
|artificial intelligence        |feature extraction              |159         |
|feature extraction             |feature extraction              |159         |
|computer science               |quality of service              |156         |
|computer science               |satisfiability                  |155         |
|computer science               |feature

In [ ]:
# Query 8: WHERE, Nested Query (i.e., 2-step Queries), GROUP BY

author_name = 'Hao Wang'

sub_nested_query = df_aut \
    .filter(col('name') == author_name) \
    .select('author_id') \
    .rdd.flatMap(lambda x: x) \
    .collect()

nested_query = df_aff \
    .filter(col('author_id').isin(sub_nested_query)) \
    .filter(col('organization') != 'null')

df_papers \
    .join(nested_query, 'paper_id') \
    .select('paper_id',
            'organization',
            explode('fos').alias('field_of_study')) \
    .groupBy('field_of_study') \
    .agg(collect_set('organization').alias('organization')) \
    .orderBy('field_of_study') \
    .show(14, truncate=80)

In [ ]:
# Query 9: WHERE, GROUP BY, HAVING, 1 JOIN

from pyspark.sql.functions import collect_set, concat, size

df_journals \
    .withColumnRenamed('venue', 'venueJournals') \
    .filter((col('volume')) > 10) \
    .join(df_books, df_books.publisher == df_journals.publisher, 'inner') \
    .drop(df_journals.publisher) \
    .withColumnRenamed('venue', 'venueBooks') \
    .select('venueBooks',
            'venueJournals',
            'publisher') \
    .dropDuplicates(['venueBooks',
                     'venueJournals',
                     'publisher']) \
    .groupBy('publisher') \
    .agg(collect_set('venueBooks').alias('books'),
         collect_set('venueJournals').alias('journals')) \
    .withColumn('total_publications_per_publisher', concat('books', 'journals')) \
    .filter(size('total_publications_per_publisher') > '500') \
    .select('publisher',
            'total_publications_per_publisher') \
    .show(truncate=50)

In [ ]:
# Query 10: WHERE, GROUP BY, HAVING, 2 JOINs
# Retrieve authors who worked for at least 3 different organizations and have published at least 3 papers with at least 5 fos and 5 references each

from pyspark.sql.functions import approx_count_distinct

df_papers \
    .filter((size(col('fos')) >= 5) &
            (size(col('references')) >= 5)) \
    .join(df_aff, df_papers.paper_id == df_aff.paper_id, 'inner') \
    .drop(df_aff.paper_id) \
    .join(df_aut, df_aff.author_id == df_aut.author_id, 'inner') \
    .drop(df_aff.author_id) \
    .groupBy('author_id') \
    .agg(count('paper_id').alias('papers_count'),
         approx_count_distinct('organization').alias('organizations_count'),
         collect_set('name').alias('name')) \
    .filter((size('name') == 1) &
            (col('papers_count') >= 3) &
            (col('organizations_count') >= 3)) \
    .orderBy(col('papers_count').desc(),
             col('organizations_count').desc()) \
    .select(explode('name').alias('name'),
            'papers_count',
            'organizations_count') \
    .show(5)